In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# import libraries for reading data, exploring and plotting
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
# library for train test split
from sklearn.model_selection import train_test_split
# deep learning libraries for text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Modeling 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional
import string
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")
# from google.colab import drive
# drive.mount('/content/drive')
def text_preprocess(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()

    # Now just remove any stopwords and non alphabets
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]
    return nostop

messages = pd.read_csv("super23.csv", encoding='latin-1')
# messages = pd.read_excel('punny.xlsx')
messages.columns =["label", "message"]

print(messages.head(5))
# print(messages.describe())

   label                                            message
0      1  YOU HAVE WON! As a valued Vodafone customer ou...
1      1  Guinness Record! World?s most pierced women! C...
2      1  U have a secret admirer. REVEAL who thinks U R...
3      1  U have a secret admirer who is looking 2 make ...
4      1  FREE for 1st week! No1 Nokia tone 4 ur mob eve...


In [ ]:
messages_train = messages.loc[0:65736, :]
print (messages_train.tail(3))

messages_train["message"] = messages_train["message"].apply(text_preprocess)
messages_train["message"] = messages_train["message"].agg(lambda x: ' '.join(map(str, x)))
# train_msg = messages_train["message"]
msg_label = messages_train["label"]

       label                                            message
65733      0             Are u struggling to access some apps. 
65734      0  Hey dude, could u please put my leave tomorrow...
65735      0  hope u received my email, you can put ur own c...


In [ ]:

# Split data into train and test
train_msg, test_msg, train_labels, test_labels = train_test_split(messages_train["message"], msg_label, test_size=0.2, random_state=434)

# Defining pre-processing hyperparameters
max_len = 50 
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>" 
vocab_size = 500

tokenizer = Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_msg)

# Get the word_index 
word_index = tokenizer.word_index

# check how many words 
tot_words = len(word_index)
print('There are %s unique tokens in training data. ' % tot_words)

# Sequencing and padding on training and testing 
training_sequences = tokenizer.texts_to_sequences(train_msg)
training_padded = pad_sequences (training_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type )
testing_sequences = tokenizer.texts_to_sequences(test_msg)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

# Shape of train tensor
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

# Before padding
print("Before Padding: ", len(training_sequences[0]), len(training_sequences[1]))

# After padding
print("After Padding: ",len(training_padded[0]), len(training_padded[1]))

There are 55976 unique tokens in training data. 
Shape of training tensor:  (52588, 50)
Shape of testing tensor:  (13148, 50)
Before Padding:  8 13
After Padding:  50 50


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
vocab_size = 14000 # As defined earlier
embeding_dim = 16
drop_value = 0.2 # dropout

* **LSTM MODEL**

In [ ]:
from keras.layers import Flatten
#LSTM hyperparameters
n_lstm = 140
drop_lstm =0.5

#LSTM Spam detection architecture
model1 = Sequential()
model1.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model1.add(Dropout(0.5))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True))
model1.add(Flatten())
model1.add(Dense(1, activation='sigmoid'))

model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

num_epochs = 50
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model1.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels),callbacks =[early_stop], verbose=0)

# Model performance on test data 
result = model1.evaluate(testing_padded, test_labels)

loss, accuracy = model1.evaluate(testing_padded, test_labels)
# loss, accuracy, f1_score, precision, recall = model1.evaluate(testing_padded, test_labels)
print ("Result: ", result)
print ("loss, accuracy: ", loss, accuracy)

# # Create a dataframe
# metrics = pd.DataFrame(history.history)
# # Rename column
# metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy',
#                          'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
# def plot_graphs1(var1, var2, string):
#     metrics[[var1, var2]].plot()
#     plt.title('LSTM Model: Training and Validation ' + string)
#     plt.xlabel ('Number of epochs')
#     plt.ylabel(string)
#     plt.legend([var1, var2])
# plot_graphs1('Training_Loss', 'Validation_Loss', 'loss')
# plot_graphs1('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

In [ ]:
pred_lbl = (model1.predict(testing_padded) > 0.5).astype("int32") #decode(en, x_test)
pd.DataFrame(pred_lbl).to_csv('output1.csv', index=False)

import pandas as pd
messages = pd.read_csv("output1.csv", encoding='latin-1')
messages.columns = ["labels"]
print (messages.tail(3))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_labels, messages["labels"]))
print(metrics.confusion_matrix(test_labels, messages["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(test_labels, messages["labels"])))


* **Bidirectional LSTM MODEL**

In [ ]:
from keras.layers import Flatten
#LSTM hyperparameters
n_lstm = 140
drop_lstm =0.5

# Biderectional LSTM Spam detection architecture
model2 = Sequential()
model2.add(Embedding(vocab_size, embeding_dim, input_length=max_len))
model2.add(Dropout(0.5))
model2.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm, return_sequences=True)))
model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Training
num_epochs = 10
early_stop = EarlyStopping(monitor='val_loss', patience=2)
history = model2.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels),callbacks =[early_stop], verbose=2)

# Model performance on test data 
result = model2.evaluate(testing_padded, test_labels)

# loss, accuracy, f1_score, precision, recall = model.evaluate(testing_padded, test_labels)
print ("Result: ", result)
# print ("loss, accuracy, f1_score, precision, recall: ", loss, accuracy, f1_score, precision, recall)

# # Create a dataframe
# metrics = pd.DataFrame(history.history)
# # Rename column
# metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy',
#                          'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
# def plot_graphs1(var1, var2, string):
#     metrics[[var1, var2]].plot()
#     plt.title('BiLSTM Model: Training and Validation ' + string)
#     plt.xlabel ('Number of epochs')
#     plt.ylabel(string)
#     plt.legend([var1, var2])
# # Plot
# plot_graphs1('Training_Loss', 'Validation_Loss', 'loss')
# plot_graphs1('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

In [ ]:
pred_lbl2 = (model2.predict(testing_padded) > 0.5).astype("int32") #decode(en, x_test)
pd.DataFrame(pred_lbl2).to_csv('output2.csv', index=False)

import pandas as pd
messages2 = pd.read_csv("output2.csv", encoding='latin-1')
messages2.columns = ["labels"]
print (messages2.tail(3))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_labels, messages2["labels"]))
print(metrics.confusion_matrix(test_labels, messages2["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(test_labels, messages2["labels"])))

**CNN MODEL**

In [ ]:
from keras.layers import Conv1D, MaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
embedding_size = 100

model22 = Sequential()
model22.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
model22.add(Conv1D(128, 3, activation='relu'))
model22.add(MaxPool1D(3))
model22.add(Dropout(0.2))
model22.add(Conv1D(128, 3, activation='relu'))
model22.add(GlobalMaxPooling1D())
model22.add(Dropout(0.2))
model22.add(Dense(64, activation='relu'))
model22.add(Dropout(0.2))
model22.add(Dense(32, activation='relu'))
model22.add(Dropout(0.2))
model22.summary()
model22.add(Dense(1, activation='sigmoid'))

model22.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])

# Training
num_epochs = 10
early_stop = EarlyStopping(monitor='val_loss', patience=2)
result22 = model22.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels),callbacks =[early_stop], verbose=2)

# Model performance on test data 
result = model22.evaluate(testing_padded, test_labels)

# loss, accuracy, f1_score, precision, recall = model.evaluate(testing_padded, test_labels)
print ("Result: ", result)
# print ("loss, accuracy, f1_score, precision, recall: ", loss, accuracy, f1_score, precision, recall)

In [ ]:
pred_lbl2 = (model22.predict(testing_padded) > 0.5).astype("int32") #decode(en, x_test)
pd.DataFrame(pred_lbl2).to_csv('output3.csv', index=False)

import pandas as pd
messages2 = pd.read_csv("output3.csv", encoding='latin-1')
messages2.columns = ["labels"]
print (messages2.tail(3))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_labels, messages2["labels"]))
print(metrics.confusion_matrix(test_labels, messages2["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(test_labels, messages2["labels"])))

* **CNN with Glove Model**

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
from keras.layers import Conv1D, MaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

embedding_size = 100

word2vec = {}
f = open('glove.6B.100d.txt', encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
    
print(len(word2vec))

embedding_matrix = np.zeros((vocab_size, embedding_size))

for word,i in  tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
         # words not in the glove will be set to zero   
            embedding_matrix[i] = embedding_vector

n_epochs = 50
model4 = Sequential()
model4.add(Embedding(input_dim=vocab_size, output_dim=embedding_size,weights = [embedding_matrix], input_length=max_len, trainable=True))
model4.add(Conv1D(128, 3, activation='relu'))
model4.add(MaxPool1D(3))
model4.add(Dropout(0.2))
model4.add(Conv1D(128, 3, activation='relu'))
model4.add(GlobalMaxPooling1D())
model4.add(Dropout(0.2))
model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(32, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(16, activation='relu'))
model4.add(Dropout(0.2))
model4.summary()
model4.add(Dense(1, activation='sigmoid'))

model4.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
save_best = ModelCheckpoint('SMS.hdf', save_best_only=True, monitor='val_acc', mode='max')
result4 = model4.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(testing_padded, test_labels),callbacks =[early_stop], verbose=2)
# result4 = model4.fit(np.array(X_train), np.array(y_train), batch_size = batch_size, epochs=n_epochs, validation_split=0.2, verbose=1,  callbacks=[save_best])
eval_ = model4.evaluate(np.array(testing_padded), np.array(test_labels))
print(eval_[0], eval_[1]) # loss / accuracy

In [ ]:
pred_lbl2 = (model4.predict(testing_padded) > 0.5).astype("int32") #decode(en, x_test)
pd.DataFrame(pred_lbl2).to_csv('output4.csv', index=False)

import pandas as pd
messages2 = pd.read_csv("output4.csv", encoding='latin-1')
messages2.columns = ["labels"]
print (messages2.tail(3))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(test_labels, messages2["labels"]))
print(metrics.confusion_matrix(test_labels, messages2["labels"]))

# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(test_labels, messages2["labels"])))

other articles
https://towardsdatascience.com/spam-detection-in-emails-de0398ea3b48
https://towardsdatascience.com/deep-learning-techniques-for-text-classification-78d9dc40bf7c

https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model